In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
prefix = 'wm5-normalize1'
data = pd.read_csv('../../applications/%s/data/%s_with_clustered_label.csv' % (prefix,prefix))
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2466 entries, 0 to 2465
Data columns (total 7 columns):
id                 2466 non-null int64
week               2466 non-null object
TA_video           2466 non-null float64
TA_text            2466 non-null float64
Teacher_video      2466 non-null float64
Teacher_text       2466 non-null float64
clustered_label    2466 non-null float64
dtypes: float64(5), int64(1), object(1)
memory usage: 134.9+ KB


In [257]:
data[(data['clustered_label']>=0.662 )& (data['clustered_label']< 0.663 )]

,ReportDate,Customer,Type,ItemShortName,Brand,Sales,AWU_Avial_ratio,Backlog_Avail_ratio,Actual_AWU,Avail,BL<=9WKs,Backlog,DCOH,Hub_OH,TTL_OH,FCST_M,clustered_label


In [258]:
print('===================All===================')
print('mean:',round(np.mean(data['Actual_AWU']), 5))
print('std:',round(np.std(data['Actual_AWU']), 5))

===================All===================
mean: 0.00794
std: 0.03553


In [259]:
import numpy as np
import pandas as pd
from pandas import ExcelWriter
import openpyxl
from pymongo import MongoClient
import argparse
import os

def layers(name):
    source_path = name.replace('-item-seq','')
    # init an array to store every layers 
    layer = []
    max_layer = 1
    # list all files in /output folder
    for file in os.listdir('../../applications/%s/GHSOM/output/%s' % (source_path,name)):
        # get all .unit files in /output folder
        if file.endswith('.unit'):
            
            # get file path
            unit_file_path = os.path.join('../../applications/%s/GHSOM/output/%s/' % (source_path,name), file)
            # print(unit_file_path)

            # get attr from content
            text_file = open(unit_file_path).read().split()

            # get each layer dimension info
            x_dim = int(text_file[text_file.index('$XDIM')+1])
            y_dim = int(text_file[text_file.index('$YDIM')+1])
            # print('$XDIM:',text_file[text_file.index('$XDIM')+1])
            # print('$YDIM:',text_file[text_file.index('$YDIM')+1])

            # first layer
            if 'lvl' not in file:
                layer.append(x_dim*y_dim)
            # other layers
            else:
                layer_index = int(file.split('lvl')[1][0])
                max_layer = layer_index if max_layer < layer_index else max_layer
                
                if len(layer) != layer_index:
                    layer.append(x_dim*y_dim)
                else:
                    if (x_dim*y_dim) > layer[layer_index-1]:
                        layer[layer_index-1] = x_dim*y_dim
    print('layer:',layer)
    print('max_layer:',max_layer)
    
    layer_number = layer.copy()
    number_of_digits = [0] * len(layer_number)

    for i in range(len(layer_number)):
        digit = 1
        while True:
            if (layer_number[i] // 10) != 0 :
                digit = digit + 1 
                layer_number[i] = layer_number[i] // 10
            else:
                break
        number_of_digits[i] = digit
    print('number_of_digits:',number_of_digits)
    return layer,max_layer,number_of_digits

In [260]:
layer,max_layer,number_of_digits = layers(prefix)


layer: [6, 6]
max_layer: 2
number_of_digits: [1, 1]


In [261]:
data['clustered_label_integer'] = data['clustered_label']*pow(10,sum(number_of_digits))
data['clustered_label_integer'] = data['clustered_label_integer'].round().astype('int64')
clusters = data['clustered_label_integer'].unique()
print('clusters:',len(clusters))
clusters.sort()
clusters

clusters: 11


array([10, 20, 30, 40, 50, 61, 62, 63, 64, 65, 66], dtype=int64)

In [262]:
# number_of_digits.append(1)
print(np.cumsum(number_of_digits)-1)
# number_of_digits_reverse = 
number_of_digits_reverse = np.flip(np.cumsum(number_of_digits)-1, 0)
number_of_digits_reverse

[0 1]


array([1, 0], dtype=int32)

In [263]:
all_clusters = []
unique_cluster = []
for i in range(0,len(number_of_digits_reverse)):
    layer_clusters = []
#     print('number:',pow(10,number_of_digits_reverse[i]))
    for cluster in clusters:
#         cluster = cluster*pow(10,number_of_digits_reverse[i])
#         print('%s / %s = %s' % (cluster,pow(10,number_of_digits_reverse[i]-1),cluster % (pow(10,number_of_digits_reverse[i]-1))))
        if cluster % pow(10,number_of_digits_reverse[i]+1) != 0:
#             print(round(cluster)/pow(10,number_of_digits_reverse[i]))
            c = round(cluster / pow(10,number_of_digits_reverse[i])) * pow(10,number_of_digits_reverse[i])
            layer_clusters.append(c)
        unique_cluster.append(round(cluster / pow(10,number_of_digits_reverse[i])) * pow(10,number_of_digits_reverse[i]))
    all_clusters.append(np.unique(layer_clusters))

In [264]:
# all_clusters = np.unique(all_clusters)
all_clusters = np.array(all_clusters)
all_clusters

array([array([10., 20., 30., 40., 50., 60., 70.]),
       array([61., 62., 63., 64., 65., 66.])], dtype=object)

In [265]:
df_data = []
clusters_data = []
for i in range(0, len(number_of_digits_reverse)):
# for i in range(1, 2):
    print('Layer %s' % str(i+1))
    print(all_clusters[i])
    for j in all_clusters[i]:
        clusters_data = data[(data['clustered_label_integer'] >= j) & (data['clustered_label_integer'] < j + pow(10,len(number_of_digits_reverse)-1-i))]
        print(clusters_data)
        mean = np.mean(clusters_data['Actual_AWU'])
        std = np.std(clusters_data['Actual_AWU'])
        print('mean:',round(mean, 5))
        print('std:',round(std, 5))
        df_data.append([j,round(mean, 5),round(std, 5)])

Layer 1
[10. 20. 30. 40. 50. 60. 70.]
             ReportDate  Customer       Type ItemShortName    Brand  \
553      2016/9/4 17:40   10759.0  OverStock     TC7SH08FU  TOSHIBA   
818     2016/8/29 00:03   10759.0  OverStock     TC7SH08FU  TOSHIBA   
1033    2016/8/19 17:25   10759.0  OverStock     TC7SH08FU  TOSHIBA   
1246    2016/8/12 18:06   10759.0  OverStock     TC7SH08FU  TOSHIBA   
1459     2016/8/6 11:33   10759.0  OverStock     TC7SH08FU  TOSHIBA   
1739     2016/8/5 17:52   10759.0  OverStock     TC7SH08FU  TOSHIBA   
6664    2016/11/6 20:27   10759.0     Normal     TC7SH08FU  TOSHIBA   
7068   2016/10/31 11:24   10759.0  OverStock     TC7SH08FU  TOSHIBA   
7352   2016/10/24 07:45   10759.0  OverStock     TC7SH08FU  TOSHIBA   
7761   2016/10/10 12:48   10759.0  OverStock     TC7SH08FU  TOSHIBA   
8143    2016/10/2 22:51   10759.0  OverStock     TC7SH08FU  TOSHIBA   
8596     2017/1/6 02:05    1171.0  OverStock     T2N7002BK  TOSHIBA   
8599     2017/1/6 02:05    1171.0  Over

[3142 rows x 18 columns]
mean: 0.00324
std: 0.00306
             ReportDate  Customer       Type        ItemShortName      Brand  \
79     2016/11/28 03:00   10759.0     Normal            AXG110144  PANASONIC   
119    2016/11/28 03:00   10759.0     Normal       MP20073DH-LF-Z        MPS   
123    2016/11/28 03:00   10759.0     Normal           NB650AGL-Z        MPS   
141    2016/11/28 03:00   10759.0  OverStock       SKY13563-670LF   SKYWORKS   
165    2016/11/28 03:00   10759.0     Normal             SPR-P110       PTTC   
238    2016/11/28 03:00   10759.0     Normal             RTC8612H   RICHWAVE   
323      2016/9/4 17:40   10759.0     Normal  12-22A/Y2G6C-B30/2C  EVERLIGHT   
343      2016/9/4 17:40   10759.0     Normal       SKY13562-670LF   SKYWORKS   
345      2016/9/4 17:40   10759.0     Normal       SKY19245-686LF   SKYWORKS   
354      2016/9/4 17:40   10759.0     Normal        SMD1812P110TF       PTTC   
372      2016/9/4 17:40   10759.0     Normal            TC7SH32FU   

In [266]:
# all_clusters.shape

In [267]:
len(np.unique(unique_cluster))

13

In [268]:
# data[(data['clustered_label_integer'] >= 66421) & (data['clustered_label_integer'] < 66422)]

In [269]:
# df_new = pd.DataFrame(np.unique(unique_cluster).astype(int), columns = ["cluster"])

In [270]:
# df_new

In [271]:
df_new = pd.DataFrame(df_data, columns = ['cluster','mean','std'])
df_new = df_new.dropna()
df_new

,cluster,mean,std
0,10.0,0.34577,0.16437
1,20.0,0.12129,0.10229
2,30.0,0.03305,0.02713
3,40.0,0.13514,0.09765
4,50.0,0.03242,0.02404
5,60.0,0.00177,0.00421
7,61.0,0.00043,0.00087
8,62.0,0.00466,0.00328
9,63.0,0.00324,0.00306
10,64.0,0.02148,0.00801


In [272]:
df_new.to_csv('../../applications/%s/data/clusters_info.csv' % prefix)